# Managing assignment files

Distributing assignments to students and collecting them can be a logistical nightmare. If you are running nbgrader on a server, some of this pain can be relieved by relying on nbgrader's built-in functionality for releasing and collecting assignments on the instructor's side, and fetching and submitting assignments on the student's side.

## Setting up the exchange

After an assignment has been created using `nbgrader assign`, the instructor must actually release that assignment to students. If the class is being taught on a single filesystem, then the instructor may use `nbgrader release` to copy the assignment files to a shared location on the filesystem for students to then download.

First, we must specify a few configuration options. To do this, we'll create a `nbgrader_config.py` file that will get automatically loaded when we run `nbgrader`:

In [1]:
%%file nbgrader_config.py

c = get_config()

c.Exchange.course_id = "example_course"
c.Exchange.root = "/tmp/exchange"

Writing nbgrader_config.py


In the config file, we've specified the "exchange" directory to be `/tmp/exchange`. This directory must exist before running `nbgrader`, and it *must* be readable and writable by all users, so we'll first create it and configure the appropriate permissions:

In [2]:
%%bash

# remove existing directory, so we can start fresh for demo purposes
rm -rf /tmp/exchange

# create the exchange directory, with write permissions for everyone
mkdir /tmp/exchange
chmod ugo+rw /tmp/exchange

## Releasing assignments

### From the formgrader

Using the formgrader extension, you may release assignments by clicking on the "release" button:

![](images/manage_assignments5.png)

**Note** that for the "release" button to become available, the `course_id` option must be set in `nbgrader_config.py`.
Once completed, you will see a pop-up window with log output:

![](images/release_assignment.png)

If you decide you want to "un-release" an assignment, you may do so by clicking again on the "release" button (which is now an "x"). **However, note that students who have already downloaded the assignment will still have access to their downloaded copy. Unreleasing an assignment only prevents more students from downloading it.**

![](images/manage_assignments6.png)

### From the command line

Now that we have the directory created, we can actually run `nbgrader release` (and as with the other nbgrader commands for instructors, this must be run from the root of the course directory):

In [3]:
%%bash

nbgrader release "ps1"

[ReleaseApp | INFO] Source: /Users/jhamrick/project/tools/nbgrader/nbgrader/docs/source/user_guide/release/./ps1
[ReleaseApp | INFO] Destination: /tmp/exchange/example_course/outbound/ps1
[ReleaseApp | INFO] Released as: example_course ps1


Finally, you can verify that the assignment has been appropriately released by running the `nbgrader list` command:

In [4]:
%%bash

nbgrader list

[ListApp | INFO] Released assignments:
[ListApp | INFO] example_course ps1


Note that there should only ever be *one* instructor who runs the `nbgrader release` and `nbgrader collect` commands (and there should probably only be one instructor -- the same instructor -- who runs `nbgrader assign`, `nbgrader autograde` and the formgrader as well). However this does not mean that only one instructor can do the grading, it just means that only one instructor manages the assignment files. Other instructors can still perform grading by accessing the notebook where the formgrader is running.

## Fetching assignments

From the student's perspective, they can list what assignments have been released, and then fetch a copy of the assignment to work on. First, we'll create a temporary directory to represent the student's home directory:

In [5]:
%%bash

# remove the fake student home directory if it exists, for demo purposes
rm -rf /tmp/student_home

# create the fake student home directory and switch to it
mkdir /tmp/student_home

If you are not using the default exchange directory (as is the case here), you will additionally need to provide your students with a configuration file that sets the appropriate directory for them:

In [6]:
%%file /tmp/student_home/nbgrader_config.py

c = get_config()
c.Exchange.root = '/tmp/exchange'
c.Exchange.course_id = "example_course"

Writing /tmp/student_home/nbgrader_config.py


### From the notebook dashboard

![](images/assignment_list_released.png)

The image above shows that there has been one assignment released ("ps1") for the class "example_course". To get this assignment, students can click the "Fetch" button (analogous to running `nbgrader fetch ps1 --course example_course`. **Note: this assumes nbgrader is always run from the root of the notebook server, which on JupyterHub is most likely the root of the user's home directory.**

After the assignment is fetched, it will appear in the list of "Downloaded assignments":

![](images/assignment_list_downloaded.png)

Students can click on the name of the assignment to expand it and see all the notebooks in the assignment:

![](images/assignment_list_downloaded_expanded.png)

Clicking on a particular notebook will open it in a new tab in the browser.

### From the command line

From the student's perspective, they can see what assignments have been released using `nbgrader list`, and passing the name of the class:

In [7]:
%%bash
export HOME=/tmp/student_home && cd $HOME

nbgrader list

[ListApp | INFO] Released assignments:
[ListApp | INFO] example_course ps1


They can then fetch an assignment for that class using `nbgrader fetch` and passing the name of the class and the name of the assignment:

In [8]:
%%bash
export HOME=/tmp/student_home && cd $HOME

nbgrader fetch "ps1"

[FetchApp | INFO] Source: /tmp/exchange/example_course/outbound/ps1
[FetchApp | INFO] Destination: /private/tmp/student_home/ps1
[FetchApp | INFO] Fetched as: example_course ps1


Note that running `nbgrader fetch` copies the assignment files from the exchange directory to the local directory, and therefore can be used from any directory:

In [9]:
%%bash

ls -l "/tmp/student_home/ps1"

total 40
-rw-r--r--  1 jhamrick  wheel  5733 Oct 20 16:27 jupyter.png
-rw-r--r--  1 jhamrick  wheel  7884 Oct 20 16:27 problem1.ipynb
-rw-r--r--  1 jhamrick  wheel  2258 Oct 20 16:27 problem2.ipynb


Additionally, the `nbgrader fetch` (as well as `nbgrader submit`) command also does not rely on having access to the nbgrader database -- the database is only used by instructors.

## Submitting assignments

### From the notebook dashboard

After students have worked on the assignment for a while, but before submitting, they can validate that their notebooks pass the tests by clicking the "Validate" button (analogous to running `nbgrader validate`). If any tests fail, they will see a warning:

![](images/assignment_list_validate_failed.png)

If there are no errors, they will see that the validation passes:

![](images/assignment_list_validate_succeeded.png)

Once students have validated all the notebooks, they can click the "Submit" button to submit the assignment (analogous to running `nbgrader submit ps1 --course example_course`). Afterwards, it will show up in the list of submitted assignments (and also still in the list of downloaded assignments):

![](images/assignment_list_submitted.png)

Students may submit an assignment as many times as they'd like. All copies of a submission will show up in the submitted assignments list, and when the instructor collects the assignments, they will get the most recent version of the assignment:

![](images/assignment_list_submitted_again.png)

Similarly, if the ``strict`` option (in the student's ``nbgrader_config.py`` file) is set to ``True``, the students will not be able to submit an assignment with missing notebooks (for a given assignment):

![](images/assignment_list_submit_error.jpg)

### From the command line

First, as a reminder, here is what the student's `nbgrader_config.py` file looks like:

In [10]:
%%bash

cat /tmp/student_home/nbgrader_config.py


c = get_config()
c.Exchange.root = '/tmp/exchange'
c.Exchange.course_id = "example_course"

After working on an assignment, the student can submit their version for grading using `nbgrader submit` and passing the name of the assignment and the name of the class:

In [11]:
%%bash
export HOME=/tmp/student_home && cd $HOME

nbgrader submit "ps1"

[SubmitApp | INFO] Source: /private/tmp/student_home/ps1
[SubmitApp | INFO] Destination: /tmp/exchange/example_course/inbound/jhamrick+ps1+2017-10-20 23:27:56.608573 UTC
[SubmitApp | INFO] Submitted as: example_course ps1 2017-10-20 23:27:56.608573 UTC


Note that "the name of the assignment" really corresponds to "the name of a folder". It just happens that, in our current directory, there is a folder called "ps1":

In [12]:
%%bash
export HOME=/tmp/student_home && cd $HOME

ls -l "/tmp/student_home"

total 8
drwxr-xr-x  3 jhamrick  wheel  102 Oct 20 16:27 Library
-rw-r--r--  1 jhamrick  wheel   91 Oct 20 16:27 nbgrader_config.py
drwxr-xr-x  5 jhamrick  wheel  170 Oct 20 16:27 ps1


Students can see what assignments they have submitted using `nbgrader list --inbound`:

In [13]:
%%bash
export HOME=/tmp/student_home && cd $HOME

nbgrader list --inbound

[ListApp | INFO] Submitted assignments:
[ListApp | INFO] example_course jhamrick ps1 2017-10-20 23:27:56.608573 UTC


Importantly, students can run `nbgrader submit` as many times as they want, and all submitted copies of the assignment will be preserved:

In [14]:
%%bash
export HOME=/tmp/student_home && cd $HOME

nbgrader submit "ps1"

[SubmitApp | INFO] Source: /private/tmp/student_home/ps1
[SubmitApp | INFO] Destination: /tmp/exchange/example_course/inbound/jhamrick+ps1+2017-10-20 23:27:59.884785 UTC
[SubmitApp | INFO] Submitted as: example_course ps1 2017-10-20 23:27:59.884785 UTC


We can see all versions that have been submitted by again running `nbgrader list --inbound`:

In [15]:
%%bash
export HOME=/tmp/student_home && cd $HOME

nbgrader list --inbound

[ListApp | INFO] Submitted assignments:
[ListApp | INFO] example_course jhamrick ps1 2017-10-20 23:27:56.608573 UTC
[ListApp | INFO] example_course jhamrick ps1 2017-10-20 23:27:59.884785 UTC


Note that the `nbgrader submit` (as well as `nbgrader fetch`) command also does not rely on having access to the nbgrader database -- the database is only used by instructors.

``nbgrader`` requires that the submitted notebook names match the released notebook names for each assignment. For example if a student were to rename one of the given assignment notebooks:

In [16]:
%%bash
export HOME=/tmp/student_home && cd $HOME

# assume the student renamed the assignment file
mv ps1/problem1.ipynb ps1/myproblem1.ipynb

nbgrader submit "ps1"

[SubmitApp | INFO] Source: /private/tmp/student_home/ps1
[SubmitApp | INFO] Destination: /tmp/exchange/example_course/inbound/jhamrick+ps1+2017-10-20 23:28:02.912676 UTC
[SubmitApp | WARNING] Possible missing notebooks and/or extra notebooks submitted for assignment ps1:
    Expected:
    	problem1.ipynb: MISSING
    	problem2.ipynb: FOUND
    Submitted:
    	myproblem1.ipynb: EXTRA
    	problem2.ipynb: OK
[SubmitApp | INFO] Submitted as: example_course ps1 2017-10-20 23:28:02.912676 UTC


By default this assignment will still be submitted however only the "FOUND" notebooks (for the given assignment) can be ``autograded`` and will appear on the ``formgrade`` extension. "EXTRA" notebooks will not be ``autograded`` and will not appear on the ``formgrade`` extension.

To ensure that students cannot submit an assignment with missing notebooks (for a given assignment) the ``strict`` option, in the student's ``nbgrader_config.py`` file, can be set to ``True``:

In [17]:
%%file /tmp/student_home/nbgrader_config.py

c = get_config()
c.Exchange.root = '/tmp/exchange'
c.Exchange.course_id = "example_course"
c.ExchangeSubmit.strict = True

Overwriting /tmp/student_home/nbgrader_config.py


In [18]:
%%bash
export HOME=/tmp/student_home && cd $HOME

nbgrader submit "ps1"

[SubmitApp | INFO] Source: /private/tmp/student_home/ps1
[SubmitApp | INFO] Destination: /tmp/exchange/example_course/inbound/jhamrick+ps1+2017-10-20 23:28:04.439223 UTC
[SubmitApp | CRITICAL] Assignment ps1 not submitted. There are missing notebooks for the submission:
    Expected:
    	problem1.ipynb: MISSING
    	problem2.ipynb: FOUND
    Submitted:
    	myproblem1.ipynb: EXTRA
    	problem2.ipynb: OK
[SubmitApp | ERROR] nbgrader submit failed


## Collecting assignments

First, as a reminder, here is what the instructor's `nbgrader_config.py` file looks like:

In [19]:
%%bash

cat nbgrader_config.py


c = get_config()

c.Exchange.course_id = "example_course"
c.Exchange.root = "/tmp/exchange"

### From the formgrader

From the formgrader extension, we can collect submissions by clicking on the "collect" button:

![](images/manage_assignments7.png)

As with releasing, this will display a pop-up window when the operation is complete, telling you how many submissions were collected:

![](images/collect_assignment.png)

From here, you can click on the number of submissions to grade the collected submissions:

![](images/manage_assignments8.png)

### From the command line

After students have submitted their assignments, the instructor can view what has been submitted with `nbgrader list --inbound`:

In [20]:
%%bash

nbgrader list --inbound

[ListApp | INFO] Submitted assignments:
[ListApp | INFO] example_course jhamrick ps1 2017-10-20 23:27:56.608573 UTC
[ListApp | INFO] example_course jhamrick ps1 2017-10-20 23:27:59.884785 UTC
[ListApp | INFO] example_course jhamrick ps1 2017-10-20 23:28:02.912676 UTC


The instructor can then collect all submitted assignments with `nbgrader collect` and passing the name of the assignment (and as with the other nbgrader commands for instructors, this must be run from the root of the course directory):

In [21]:
%%bash

nbgrader collect "ps1"

[CollectApp | INFO] Processing 1 submissions of 'ps1' for course 'example_course'
[CollectApp | INFO] Collecting submission: jhamrick ps1


This will copy the student submissions to the `submitted` folder in a way that is automatically compatible with `nbgrader autograde`:

In [22]:
%%bash

ls -l submitted

total 0
drwxr-xr-x  3 jhamrick  staff  102 May 31 11:10 bitdiddle
drwxr-xr-x  3 jhamrick  staff  102 May 31 11:10 hacker
drwxr-xr-x  3 jhamrick  staff  102 Oct 20 16:28 jhamrick


Note that there should only ever be *one* instructor who runs the `nbgrader release` and `nbgrader collect` commands (and there should probably only be one instructor -- the same instructor -- who runs `nbgrader assign`, `nbgrader autograde` and the formgrader as well). However this does not mean that only one instructor can do the grading, it just means that only one instructor manages the assignment files. Other instructors can still perform grading by accessing the notebook server running the formgrader.